In [13]:
import os
import subprocess
import multiprocessing
import itertools
import pandas as pd

path= 'images'
languages = ['c', 'asm1', 'asm2']
filters = ['hsl', 'blur', 'merge']
command = {
    'blur': './runTests {language} {filter} "{src}" "{dst}"',
    'merge': './runTests {language} {filter} "{src}" "{src2}" "{dst}" {alpha}',
    'hsl': './runTests {language} {filter} "{src}" "{dst}" {h} {s} {l}'
}

opt = 0
hhssll = [(0.0, 0.0, 0.0), (99.0, 0.0, 0.0)]
alpha = [0.0, 0.14674845465465456489765465478976546548]
#manager = multiprocessing.Manager()
#dataset = manager.list([])

def worker(filt, lang):
    output = []
    rpath = os.path.join(path, filt)

    for img in os.listdir(rpath):
        rimg = os.path.join(rpath, img)

        if os.path.isfile(rimg):
            
            if filt == 'hsl':
                for (h, s, l) in hhssll:
                    comm = command[filt].format(
                            language = lang,
                            filter = filt,
                            src = rimg,
                            dst = './current.bmp',
                            h = h,
                            s = s,
                            l = l
                        )

                    delta = subprocess.check_output(comm, shell=True)
                    delta = delta.strip()

                    output.append({
                        'filename': img,
                        'language': lang,
                        'filter': filt,
                        'delta': delta,
                        'optimization': opt,
                        'h': h,
                        's': s,
                        'l': l
                    })
            elif filt == 'blur':
                comm = command[filt].format(
                        language = lang,
                        filter = filt,
                        src = rimg,
                        dst = './current.bmp'
                    )

                delta = subprocess.check_output(comm, shell=True)
                delta = delta.strip()

                output.append({
                    'filename': img,
                    'language': lang,
                    'filter': filt,
                    'delta': delta,
                    'optimization': opt
                })
            elif filt == 'merge':
                for a in alpha:
                    comm = command[filt].format(
                            language = lang,
                            filter = filt,
                            src = rimg,
                            src2 = rimg,
                            dst = './current.bmp',
                            alpha = a
                        )

                    delta = subprocess.check_output(comm, shell=True)
                    delta = delta.strip()

                    output.append({
                        'filename': img,
                        'language': lang,
                        'filter': filt,
                        'delta': delta,
                        'optimization': opt,
                        'alpha': a
                    })
                    
    dataset.append(output)
                    
jobs = []
for filt, lang in itertools.product(filters, languages):
    p = multiprocessing.Process(target=worker, args=(filt, lang))
    jobs.append(p)
    p.start()

for job in jobs:
    job.join()

In [14]:
data = []

for lst in dataset:
    data += lst
    
dst = pd.DataFrame(data)
dst.set_index(['filter', 'language'])
dst['experiment'] = dst['filename'].apply(lambda x: x.split(' ')[0])
dst['width'] = dst['filename'].apply(lambda x: x.split(' ')[1].split('x')[0])
dst['height'] = dst['filename'].apply(lambda x: x.split(' ')[1].split('x')[1])
dst.to_csv('output.csv')

In [15]:
dst

,alpha,delta,filename,filter,h,l,language,optimization,s,experiment,width,height
0,NaN,575308,random2 200x200 (1309478823).bmp,blur,NaN,NaN,asm2,3,NaN,random2,200,200
1,NaN,590660,random1 200x200 (776532036).bmp,blur,NaN,NaN,asm2,3,NaN,random1,200,200
2,NaN,1462522,fixed 320x320 (760313750).bmp,blur,NaN,NaN,asm2,3,NaN,fixed,320,320
3,NaN,504319,fixed 188x188 (1734575198).bmp,blur,NaN,NaN,asm2,3,NaN,fixed,188,188
4,NaN,592363,random2 200x200 (1076377804).bmp,blur,NaN,NaN,asm2,3,NaN,random2,200,200
5,NaN,575662,random1 200x200 (195740084).bmp,blur,NaN,NaN,asm2,3,NaN,random1,200,200
6,NaN,612654,random2 200x200 (2067038044).bmp,blur,NaN,NaN,asm2,3,NaN,random2,200,200
7,NaN,596884,random1 200x200 (1823089412).bmp,blur,NaN,NaN,asm2,3,NaN,random1,200,200
8,NaN,572714,random1 200x200 (927612902).bmp,blur,NaN,NaN,asm2,3,NaN,random1,200,200
9,NaN,599818,random1 200x200 (168002245).bmp,blur,NaN,NaN,asm2,3,NaN,random1,200,200
